In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

In [ ]:
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = False
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 300

# Dimensionality Reduction

[Matthew R. Carbone](https://www.bnl.gov/staff/mcarbone) | _Assistant Computational Scientist, Computational Science Initiative, Brookhaven National Laboratory_

In this tutorial, you will learn what dimensionality reduction means, how you can use it to your advantage in your own work, and what some of the common methods for doing dimensionality reduction are. We will then implement these methods here on some real data.

## What is dimensionality reduction?

[Dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction) is the process of transforming a dataset from a _high dimensional_ to a _low dimensional_ space. This begs the question, **what is dimension?**

## What is dimension?

- Colloquially, when we think of dimension, we think of the three spatial dimensions we live in
- More specifically, we are referring to three _degrees of freedom_ in which we can move
- A data point is a single example in a data set
- Each data point carries with it a number of properties. For example, in a data set of cars, each car will have properties like the numer of wheels, the number of doors, miles per gallon, etc.
- The number of properties each data points has is its dimensionality
- Note: properties may or may not be independent!

### Check your understanding

Let's look at the Palmer Penguins dataset we played around with in the last tutorial.

In [ ]:
!pip install palmerpenguins

In [ ]:
from palmerpenguins import load_penguins
penguins = load_penguins()

In [ ]:
penguins

What is the dimension of the Palmer Penguins dataset?

Here's a tricker example. What is the dimensionality of a "2d color image"? Hint: it's not 2!

# Principal component analysis

[Principal Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA) is a statistically rigorous method for reducing the dimensionality of a dataset. It is mostly used as a preliminary analysis and visualization tool. Here are some reference materials you can take a look at in your free time to get a better feel for what this method is and how it works!

- [Scikit Learn's decomposition reference](https://scikit-learn.org/stable/modules/decomposition.html#decompositions)
- [PCA Explained Visually with Zero Math](https://towardsdatascience.com/principal-component-analysis-pca-explained-visually-with-zero-math-1cbf392b9e7d)

## What is PCA?

PCA is arguably the simplest dimensionality reduction method. It ultimately decomposes each data point's $d$-dimensional feature vector into a $d'$-dimensional feature vector, where $d' < d$. The new "effective features" lie in a new vector space, which is a linear combination of the old one.

To get into some of the details, we have to understand the [covariance Matrix](https://en.wikipedia.org/wiki/Covariance_matrix). The Covariance Matrix $K$ is square, symmetric, positive semi-definite matrix. The diagonals $K_{ii}$ are the variances of each feature. The off-diagonals $K_{ij}$ ($i \neq j$) are the covarainces between different features. Formally, this means

$$K_{ij} = \mathrm{cov}(X_i, X_j) = \mathbb{E}[(X_i - \mathbb{E}[X_i])(X_j - \mathbb{E}[X_j])].$$

While in general $X_i$ is a random varaible, in the case of some fixed dataset, $X_i$ is usually a _feature_ of the data. Thus, for a datset of $N$ elements and $d$ features, the covariance matrix is $d \times d.$

PCA can effectively be reduced to **diagonalizing the covariance matrix**. This new diagonal form contains elements which represent the variance of each new axis, and each new axis is an eigenvector of $K.$

## "Iris" Dataset

The Iris dataset contains three different types of Iris flowers. There are 150 examples and each example has 4 attributes (4 _dimensions_). Let's see if we can use PCA to decompose the dataset from 4 dimensions into just 2! See [here](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) and [here](https://en.wikipedia.org/wiki/Iris_flower_data_set) for more details on the dataset. For now, we'll use a simple processing script to turn the dictionary dataset into a Pandas DataFrame.

In [ ]:
from sklearn import datasets

In [ ]:
iris = datasets.load_iris()
# print(iris["DESCR"])  # For more information!

In [ ]:
def process_iris(iris=iris):
    d = {
        "sepal length (cm)": iris["data"][:, 0],
        "sepal width (cm)": iris["data"][:, 1],
        "petal length (cm)": iris["data"][:, 2],
        "petal width (cm)": iris["data"][:, 3],
        "class": iris["target"]
    }
    return pd.DataFrame(d)

In [ ]:
iris_data = process_iris(iris)

In [ ]:
iris_data

Let's do PCA by hand first. Then we can find an easier way of doing it. Below, I've written a function `covariance_matrix`, which assumes an incoming matrix of shape `N` x `d`, where `N` is the number of data points and `d` is the dimensionality of each data point.

In [ ]:
def covariance_matrix(X):
    """Let X be an N x d dataset."""
    
    mu = X.mean(axis=0)  # For each feature, find the mean
    X2 = X - mu          # Subtract off the mean from each element
    N = X.shape[0]       # Total number of data points
    return (X2.T @ X2) / (N - 1)

In [ ]:
X = iris_data.iloc[:, :4].to_numpy()

We'll also want to scale our data to 0 mean and unit variance before we go forward.

In [ ]:
X_scaled = (X - X.mean(axis=0, keepdims=True)) / (1e-8 + X.std(axis=0, keepdims=True))

In [ ]:
K = covariance_matrix(X_scaled)
K

In [ ]:
# Numpy requires a different convention, hence X.T
np.allclose(K, np.cov(X_scaled.T))

Now that we have the covariance matrix, it's time to diagonalize it. What does this mean? Essentially, it means we are looking for a transformation:

$$V^T K V = K'$$

such that $K'$ is diagonal. We're not going to go into the details here, but the way to do this is via the eigenvector decomposition of the matrix. Luckily, we have packages for this. `np.linalg.eig` provies a convenience method for calculating the eigenvalues `w` and eigenvectors `V` (in matrix form) of any provided square matrix.

In [ ]:
w, V = np.linalg.eig(K)

In [ ]:
K_prime = V.T @ K @ V         # Test the above transformation!
K_prime[K_prime < 1e-14] = 0  # Convenience for visualization only

In [ ]:
K_prime

Note that the eigenvalues `w` appear in the diagonals:

In [ ]:
w

and that these eigenvectors are orthogonal:

In [ ]:
for ii, jj in itertools.combinations(range(4), 2):
    assert np.abs(np.dot(V[:, ii], V[:, jj])) < 1e-14

So what now? Well, we have the eigenvectors of the covariance matrix, but what do we do with them? It turns out that these eigenvectors actually represent the linear combination of the features of the original dataset such that the _first_ "direction" captures the most possible variance, the _second_ "direction" captures the second most possible variance, etc. We can execute this transformation via yet another simple matrix operation:

In [ ]:
X_scaled.shape

In [ ]:
V.shape

In [ ]:
principal_values = X_scaled @ V

Don't want to do all of this work? Don't worry! `scikit-learn` has you covered.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(4)
principal_values_via_sklearn = pca.fit_transform(X_scaled)

In [ ]:
# Eigenvectors are the same up to a +/- sign, which is fine
np.allclose(np.abs(principal_values), np.abs(principal_values_via_sklearn))

The key improvement that `scikit-learn` will offer is that you don't actually have to compute all eigenvectors, as doing so is expensive. There are cheaper ways to do so, and you could do something like `pca = PCA(2)`, which tells `scikit-learn` to only compute the first to principal components. This can be very useful when you're dealing with large datasets, and performing PCA is expensive.

## "Labeled Faces in the Wild" Dataset

# Real research example

We now follow along with the manuscript Torrisi _et al_ to demonstrate how a random forest was used in a somewhat recent, highly cited research work. The data we pull below is available [open access](https://data.matr.io/4/).

S. B. Torrisi, M. R. Carbone, B. A. Rohr, J. H. Montoya, Y. Ha, J. Yano, S. K. Suram & L. Hung. [Random forest machine learning models for interpretable X-ray absorption near-edge structure spectrum-property relationships.](https://www.nature.com/articles/s41524-020-00376-6) npj Comput. Mater. 6, 109 (2020).

First, we have to get the data. To do this, we use the `requests` module to directly pull the content of the webpage, and then parse that specific format (which despite the extension is not exactly JSON). It is not important to understand the particulars.

In [ ]:
import json
import requests

In [ ]:
url = "https://s3.amazonaws.com/publications.matr.io/4/deployment/data/files/spectral_data/Ti_XY.json"
r = requests.get(url)
text = r.text.split("\n")
data = [json.loads(xx) for xx in text[:-1]]

Get the inputs and outputs from this list of dictionaries.

In [ ]:
e_grid = data[0]["E"]
spectra = np.array([
    dat["mu"] for dat in data
    if dat["one_hot_coord"] is not None
])
coordinations = np.array([
    dat["coordination"] for dat in data
    if dat["one_hot_coord"] is not None
])

Like the Palmer Penguins dataset, we have reduced our classification task to a 3-class classification problem. In this case, it is the coordination number of an X-ray-absorbing atom! If you don't know what this means, don't worry too much about it. We're simply trying to classify whether or not an X-ray absorption spectrum can be used to predict this number.

In [ ]:
np.unique(coordinations)

Here are what some of the spectra look like. These are our input features, while the classes above are our targets.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 1))

for spec in spectra[::50]:
    ax.plot(e_grid, spec, color="black", alpha=0.1)

ax.set_ylabel("$\mu(E)$ / a.u.")
ax.set_yticks([])
ax.set_xlabel("$E$ / e.V.")
plt.show()
